# Encabezados de las columnas son valores

Este notebook muestra dos ejemplos de como los encabezados o nombres de las columnas muestran valores. Este tipo de "messy datasets" tienen uso práctico en dos tipos de situaciones:

1. Presentaciones
2. Registro de observaciones espaciadas regularmente en el tiempo.

## "Housekeeping"

In [1]:
%load_ext lab_black

In [2]:
import datetime
import re

import pandas as pd
import savReaderWriter as spss

## Ejemplo 1: Religion vs. Income

> Un tipo de dataset messy común son los datos tabulares diseñados para **presentación**, donde las variables forman tanto filas y columnas, y los encabezados de las columnas son valores, y no nombres de las variables.

El [Pew Research Center](http://www.pewresearch.org/) es un centro de estudios muy prolífico e influyente en investigación sobre todo tipo de aspectos de la vida en EEUU. Los siguientes ejemplos usan datos tomados del  [Religious Landscape Study](http://www.pewforum.org/religious-landscape-study/).

### Cargando la data

Los datos son entregados en un archivo de datos de SPSS. Esta es una especificación binaria con una sección de encabezado describiendo los datos, por ejemplo, que variables/columnas están incluidas y que instancias pueden tener los datos categóricos.

Cargando la "metadata" del dataset.

In [3]:
columns = ["q16", "reltrad", "income"]
encodings = {}

# For the sake of simplicity, all data cleaning operations
# are done within the for-loop for all columns.
with spss.SavHeaderReader("data/pew.sav") as pew:
    for column in columns:
        encodings[column] = {
            int(key): (
                re.sub(
                    r"\(.*\)",
                    "",
                    (
                        value.decode("iso-8859-1")
                        .replace("\x92", "'")
                        .replace(" Churches", "")
                        .replace("Less than $10,000", "<$10k")
                        .replace("10 to under $20,000", "$10-20k")
                        .replace("20 to under $30,000", "$20-30k")
                        .replace("30 to under $40,000", "$30-40k")
                        .replace("40 to under $50,000", "$40-50k")
                        .replace("50 to under $75,000", "$50-75k")
                        .replace("75 to under $100,000", "$75-100k")
                        .replace("100 to under $150,000", "$100-150k")
                        .replace("$150,000 or more", ">150k")
                    ),
                ).strip()
            )
            for (key, value) in pew.all().valueLabels[column.encode()].items()
        }

Cargando la data y preparándola tal como está presentada en el *paper*.

In [4]:
with spss.SavReader(
    "data/pew.sav", selectVars=[column.encode() for column in columns]
) as pew:
    pew = list(pew)

# Use the above encodings to map the numeric data
# to the actual labels.
pew = pd.DataFrame(pew, columns=columns, dtype=int)
for column in columns:
    pew[column] = pew[column].map(encodings[column])

for value in ("Atheist", "Agnostic"):
    pew.loc[(pew["q16"] == value), "reltrad"] = value

income_columns = [
    "<$10k",
    "$10-20k",
    "$20-30k",
    "$30-40k",
    "$40-50k",
    "$50-75k",
    "$75-100k",
    "$100-150k",
    ">150k",
    "Don't know/Refused",
]

pew = pew.groupby(["reltrad", "income"]).size().unstack("income")
pew = pew[income_columns]
pew.index.name = "religion"

### Messy Data

La siguiente celda muestra la data "sin procesar" ("cruda", raw-data) (es decir, se supone que el preprocesamiento realizado anteriormente lo realiza otra persona y al analista de datos solo se le entrega el conjunto de datos a continuación).

In [5]:
pew.shape

(18, 10)

In [6]:
pew.head(10)

income,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k,$75-100k,$100-150k,>150k,Don't know/Refused
religion,,,,,,,,,,
Agnostic,27,34,60,81,76,137,122,109,84,96
Atheist,12,27,37,52,35,70,73,59,74,76
Buddhist,27,21,30,34,33,58,62,39,53,54
Catholic,418,617,732,670,638,1116,949,792,633,1489
Don't know/refused,15,14,15,11,10,35,21,17,18,116
Evangelical Protestant,575,869,1064,982,881,1486,949,723,414,1529
Hindu,1,9,7,9,11,34,47,48,54,37
Historically Black Protestant,228,244,236,238,197,223,131,81,78,339
Jehovah's Witness,20,27,24,24,21,30,15,11,6,37


### Tidy Data

> Este dataset tiene  **tres** variables, **religion**, **income** y **frequency**. Para hacerlo *tidy*, necesitamos *"fundirlo"* (hacer un **melt**), o apilarlo. En otras palabras, necesitamos convertir columnas en filas.

`pandas` provee un método[pd.melt()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html) para *des-pivotear* el dataset.

**Notas:** `.reset_index()` transforma la columna de índice de religión en una columna de datos (`pd.melt()` lo necesita). Además, la tabla resultante se ordena implícitamente por la columna `"religión"`. Para llegar al mismo orden que en el *paper*, la tabla *fundida* se ordena explícitamente.

In [7]:
molten_pew = pd.melt(pew.reset_index(), id_vars=["religion"], value_name="frequency")

In [8]:
# Create a ordered column for the income labels.
income_dtype = pd.api.types.CategoricalDtype(income_columns, ordered=True)
molten_pew["income"] = molten_pew["income"].astype(income_dtype)
molten_pew = molten_pew.sort_values(["religion", "income"]).reset_index(drop=True)

In [9]:
molten_pew.shape

(180, 3)

In [10]:
molten_pew.head(10)

,religion,income,frequency
0,Agnostic,<$10k,27
1,Agnostic,$10-20k,34
2,Agnostic,$20-30k,60
3,Agnostic,$30-40k,81
4,Agnostic,$40-50k,76
5,Agnostic,$50-75k,137
6,Agnostic,$75-100k,122
7,Agnostic,$100-150k,109
8,Agnostic,>150k,84
9,Agnostic,Don't know/Refused,96


## Ejemplo 2: Billboard

> Otro uso común de este formato de datos es registrar observaciones espaciadas regularmente a lo largo del tiempo. Por ejemplo, el conjunto de datos de Billboard que se muestra en la Tabla 7, registra la fecha en que una canción ingresó por primera vez al Billboard Top 100. Tiene variables para **artist**, **track**, **date.entered**, **rank* * y **week**. El rango en cada semana después de que ingresa al top 100 se registra en 75 columnas,  `wk1` a `wk75`. Si una canción está en el Top 100 por menos de 75 semanas, las columnas restantes se llenan con valores faltantes. Esta forma de almacenamiento no es ordenada, pero es útil para la entrada de datos. Reduce la duplicación ya que, de lo contrario, cada canción de cada semana necesitaría su propia fila, y los metadatos de la canción, como el título y el artista, tendrían que repetirse.


### Cargando la data

Los datos vienen en un archivo CSV con columnas con nombre de número de semana de una manera engorrosa.

In [11]:
# Usage of "1st", "2nd", "3rd" should be forbidden by law :)
usecols = ["artist.inverted", "track", "time", "date.entered"] + (
    [f"x{i}st.week" for i in range(1, 76, 10) if i != 11]
    + [f"x{i}nd.week" for i in range(2, 76, 10) if i != 12]
    + [f"x{i}rd.week" for i in range(3, 76, 10) if i != 13]
    + [f"x{i}th.week" for i in range(1, 76) if (i % 10) not in (1, 2, 3)]
    + [f"x11th.week", f"x12th.week", f"x13th.week"]
)
billboard = pd.read_csv(
    "data/billboard.csv",
    encoding="iso-8859-1",
    parse_dates=["date.entered"],
    usecols=usecols,
)

billboard = billboard.assign(year=lambda x: x["date.entered"].dt.year)

# Rename the week columns.
week_columns = {
    column: ("wk" + re.sub(r"[^\d]+", "", column))
    for column in billboard.columns
    if column.endswith(".week")
}
billboard = billboard.rename(columns={"artist.inverted": "artist", **week_columns})

# Ensure the columns' order is the same as in the paper.
columns = ["year", "artist", "track", "time", "date.entered"] + [
    f"wk{i}" for i in range(1, 76)
]
billboard = billboard[columns]

# Ensure the rows' order is similar as in the paper.
# For unknown reasons the exact ordering as in the paper cannot be reconstructed.
billboard = billboard[billboard["year"] == 2000]
billboard = billboard.sort_values(["artist", "track"])

### Messy Data

De nuevo, la siguiente celda muestra los datos como si fueran realmente entregados como datos "crudos".

In [12]:
billboard.shape

(267, 80)

In [13]:
billboard.head(10)

,year,artist,track,time,date.entered,wk1,wk2,wk3,wk4,wk5,...,wk66,wk67,wk68,wk69,wk70,wk71,wk72,wk73,wk74,wk75
246,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,2000-02-26,87,82.0,72.0,77.0,87.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287,2000,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Ba...,3:15,2000-09-02,91,87.0,92.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,81,70.0,68.0,67.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,2000,3 Doors Down,Loser,4:24,2000-10-21,76,76.0,72.0,69.0,67.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,57,34.0,25.0,17.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,2000,98¡,Give Me Just One Night (Una Noche),3:24,2000-08-19,51,39.0,34.0,26.0,26.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304,2000,A*Teens,Dancing Queen,3:44,2000-07-08,97,97.0,96.0,95.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135,2000,Aaliyah,I Don't Wanna,4:15,2000-01-29,84,62.0,51.0,41.0,38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,2000,Aaliyah,Try Again,4:03,2000-03-18,59,53.0,38.0,28.0,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200,2000,"Adams, Yolanda",Open My Heart,5:30,2000-08-26,76,76.0,74.0,69.0,68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### "Tidy" Data

Como antes, el método `pd.melt()` se usa para transformar los datos desde un formato "wide" ("ancho") a uno "largo" ("long").

In [14]:
molten_billboard = pd.melt(
    billboard,
    id_vars=["year", "artist", "track", "time", "date.entered"],
    var_name="week",
    value_name="rank",
)

A diferencia de `R`, `pandas` mantiene (innecesariamente según algunos) filas para semanas donde la canción ya estaba fuera del ranking. Estas observaciones son descartadas. También, una nueva columna `"date"` es añadida, indicando cuando exactamente una canción en particular estuvo en un cierto ranking.

In [15]:
# pandas keeps "wide" variables that had missing values as rows.
molten_billboard = molten_billboard[molten_billboard["rank"].notnull()]

# Cast as integer after missing values are removed.
molten_billboard["week"] = molten_billboard["week"].map(lambda x: int(x[2:]))
molten_billboard["rank"] = molten_billboard["rank"].map(int)

# Calculate the actual week from the date of first entering the list.
molten_billboard = molten_billboard.assign(
    date=lambda x: x["date.entered"] + (x["week"] - 1) * datetime.timedelta(weeks=1)
)

# Sort rows and columns as in the paper.
molten_billboard = molten_billboard[
    ["year", "artist", "time", "track", "date", "week", "rank"]
]
molten_billboard = molten_billboard.sort_values(["artist", "track", "week"])
molten_billboard = molten_billboard.reset_index(drop=True)

Ten en cuenta que este conjunto de datos aún no está completamente `tidy`, como se explicará en el notebook 4.

In [16]:
molten_billboard.head(15)

,year,artist,time,track,date,week,rank
0,2000,2 Pac,4:22,Baby Don't Cry (Keep Ya Head Up II),2000-02-26,1,87
1,2000,2 Pac,4:22,Baby Don't Cry (Keep Ya Head Up II),2000-03-04,2,82
2,2000,2 Pac,4:22,Baby Don't Cry (Keep Ya Head Up II),2000-03-11,3,72
3,2000,2 Pac,4:22,Baby Don't Cry (Keep Ya Head Up II),2000-03-18,4,77
4,2000,2 Pac,4:22,Baby Don't Cry (Keep Ya Head Up II),2000-03-25,5,87
5,2000,2 Pac,4:22,Baby Don't Cry (Keep Ya Head Up II),2000-04-01,6,94
6,2000,2 Pac,4:22,Baby Don't Cry (Keep Ya Head Up II),2000-04-08,7,99
7,2000,2Ge+her,3:15,The Hardest Part Of Breaking Up (Is Getting Ba...,2000-09-02,1,91
8,2000,2Ge+her,3:15,The Hardest Part Of Breaking Up (Is Getting Ba...,2000-09-09,2,87
9,2000,2Ge+her,3:15,The Hardest Part Of Breaking Up (Is Getting Ba...,2000-09-16,3,92


### Guardando los datos
El dataset de bilboard ya "ordenado" ("tidy") es guardado como input para el notebook 4.

In [17]:
molten_billboard.to_csv("data/billboard_cleaned.csv", index=False)